In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

from pyspark.sql.types import *
import pyspark.sql.types as spark_types

import utils

spark = SparkSession.builder.master("spark://vm1:7077").appName("Cluster Code - Zafar").getOrCreate()

In [2]:
# Testing to see if everything is okay
commits = utils.read_csv(spark, "hdfs:/commits_new.csv")
commits.limit(10).show()

+---+---------+------------+----------+-------------------+
| id|author_id|committer_id|project_id|         created_at|
+---+---------+------------+----------+-------------------+
|  1|        1|           1|         1|2012-08-01 20:33:21|
|  2|        2|           2|         1|2012-08-01 13:25:36|
|  3|        2|           2|         1|2012-06-18 03:39:30|
|  4|        2|           2|         1|2012-06-11 07:47:16|
|  5|        2|           2|         1|2012-06-11 07:45:07|
|  6|        2|           2|         1|2012-05-07 06:00:56|
|  7|        2|           2|         1|2012-03-08 04:47:19|
|  8|        2|           2|         1|2012-03-08 04:40:43|
|  9|        2|           2|         1|2012-03-08 04:40:25|
| 10|        2|           2|         1|2012-03-08 04:24:22|
+---+---------+------------+----------+-------------------+



### Number of source & fork repositories of users (shouldn't be run again)

In [5]:
projects =  utils.read_csv(spark, "hdfs:/projects.csv", "projects_new.csv")

# This file has now been replaced with a new version
user_more =  utils.read_csv(spark, "hdfs:/user_more.csv")

# Find source repos
df4 = (
    projects
    .where((projects.deleted == 0) & (projects.forked_from.isNull()))
    .groupby("owner_id")
    .count()
    .withColumnRenamed("count", "repos_source")
    .withColumnRenamed("owner_id", "user_id")
)

# Find forks
df6 = (
    projects
    .where((projects.deleted == 0) & (projects.forked_from.isNotNull()))
    .groupby("owner_id")
    .count()
    .withColumnRenamed("count", "repos_forks")
    .withColumnRenamed("owner_id", "user_id")
)

# Join Data
df5 = user_more.join(df4, "user_id", "full").join(df6, "user_id", "full")

# Write to local directorya
df5.write.csv(
    "/user_more_2",
    mode="overwrite",
    nullValue="\\N"
)

### Issue Punchcard

In [5]:
issues = utils.read_csv(spark, "hdfs:/issues.csv")

df5 = (
    issues
    .where(
        issues.created_at.isNotNull()
    )
    .select(
        F.year('created_at').alias('year'), 
        F.month('created_at').alias('month'), 
        F.dayofmonth('created_at').alias('day'), 
        F.hour('created_at').alias('hour')
    )
   .groupBy('year', 'month', 'day', 'hour')
   .count()
)

df5.limit(10).show()

# df5.explain()
# df5.coalesce(1).write.json("hdfs:/issue_punchcard")

+----+-----+----+----+--------+
|year|month| day|hour|   count|
+----+-----+----+----+--------+
|null| null|null|null|54086297|
+----+-----+----+----+--------+



### Number of Commits of each user

In [15]:
commits = utils.read_csv(spark, "hdfs:/commits_new.csv")

res = (
    commits
    .groupby("author_id")
    .count()
    .withColumnRenamed("count", "commits_authored")
    .withColumnRenamed("author_id", "user_id")
)

res.write.csv(
    "hdfs:/user_commit_count",
    mode="overwrite",
    nullValue="\\N"
)

### Number of Commits of every project

In [16]:
# commits = utils.read_csv(spark, "hdfs:/commits_new.csv")

res = (
    commits
    .groupby("project_id")
    .count()
    .withColumnRenamed("count", "total_commits")
)

res.write.csv(
    "hdfs:/project_commit_count",
#     mode="overwrite",
    nullValue="\\N"
)

### Number of Commits on a project not authored by Owner

In [17]:
commits = utils.read_csv(spark, "hdfs:/commits_new.csv")
projects = utils.read_csv(spark, "hdfs:/projects_new.csv")

commits.createOrReplaceTempView("commits")
projects.createOrReplaceTempView("projects")

q = """
    SELECT C.project_id as project_id, COUNT(*) as total_commits_by_others
    FROM commits as C, projects as P
    WHERE C.project_id = P.id
    AND C.author_id <> P.owner_id
    GROUP BY C.project_id
"""

res = spark.sql(q)

res.write.csv(
    "hdfs:/project_commit_others_count",
#     mode="overwrite",
    nullValue="\\N"
)

### Number of commits of a user made on repositories not owned by them

In [18]:
q = """
    SELECT C.author_id as user_id, 
           COUNT(*) as total_commits_on_other_repos

    FROM commits as C, projects as P

    WHERE C.project_id = P.id
    AND C.author_id <> P.owner_id

    GROUP BY C.author_id
"""

res = spark.sql(q)

res.write.csv(
    "hdfs:/user_commit_others_count",
#     mode="overwrite",
    nullValue="\\N"
)

### "Top" Users

In [24]:
users_more = utils.read_csv(spark, "hdfs:/users_more.csv")
users = utils.read_csv(spark, "hdfs:/users.csv")

users = (
    users
    .where(
          (users.type == "USR")
        & (users.deleted == 0)
        & (users.fake == 0)
    )
    .withColumnRenamed("id", "user_id")
    .select("user_id", "login", "company")
)

user_new = users.join(users_more, "user_id", "left")

top = (
    user_new
#     .orderBy("followers", ascending=False)
    .orderBy("has_stars", ascending=False)
)

top.show(50)

+-------+-------------+--------------------+---------+---------+-------+------------+-----------+---------+---------------+------+-----+-------+--------------+
|user_id|        login|             company|following|followers|starred|repos_source|repos_forks|has_stars|contributess_to|issues|pulls|commits|commits_others|
+-------+-------------+--------------------+---------+---------+-------+------------+-----------+---------+---------------+------+-----+-------+--------------+
|   3871| sindresorhus|@avajs @chalk @ye...|      188|    15755|   2274|         974|        142|   226099|            565|  3080|  108|  26326|          8341|
|6498757| FreeCodeCamp|                null|        1|      110|      6|          95|          5|   214583|           null|     1| null|     18|          null|
| 376498|           Tj|                Apex|      175|    25827|    966|         238|         81|   114326|            138|   873| null|   4922|          3539|
| 234594|       docker|                n

### "Top" Users from India

In [34]:
users_more = utils.read_csv(spark, "hdfs:/users_more.csv")
users = utils.read_csv(spark, "hdfs:/users.csv")

users = (
    users
    .where(
          (users.type == "USR")
        & (users.deleted == 0)
        & (users.fake == 0)
        & (users.country_code == "in")
    )
    .withColumnRenamed("id", "user_id")
    .select("user_id", "login", "company", "state", "city")
)

users_more = (
    users_more
    .select("user_id", "followers", "has_stars", "contributes_to", "issues", "pulls", "commits", "commits_others")
)

user_new = (
    users
    .join(users_more, "user_id", "left")
)

top = (
    user_new
#     .orderBy("followers", ascending=False)
    .orderBy("has_stars", ascending=False)
#     .orderBy("contributes_to", ascending=False)
)

top.show(50)

+--------+-----------------+--------------------+--------------+---------+---------+---------+--------------+------+-----+-------+--------------+
| user_id|            login|             company|         state|     city|followers|has_stars|contributes_to|issues|pulls|commits|commits_others|
+--------+-----------------+--------------------+--------------+---------+---------+---------+--------------+------+-----+-------+--------------+
|   23402|          hemanth|             @paypal|          null|     null|     1209|    14032|           148|   505|   39|   5212|          1420|
| 6310255|amitshekhariitbhu|          Bobble App|         Delhi|New Delhi|      311|    10360|             7|    10| null|   1581|           509|
| 3408218|    sachinchoolur|Available for New...|     Karnataka|Bengaluru|      255|     9799|             3|     1|    1|    901|            53|
|  163695|        chinchang|           @wingify |         Delhi|    Delhi|      468|     9326|             9|   136|    2|  

In [35]:
users_more = utils.read_csv(spark, "hdfs:/users_more.csv")
users = utils.read_csv(spark, "hdfs:/users.csv")

users = (
    users
    .withColumnRenamed("id", "user_id")
#     .select("user_id", "login", "company", "state", "city")
)

users_more = (
    users_more
#     .select("user_id", "followers", "has_stars", "contributes_to", "issues", "pulls", "commits", "commits_others")
)

res = (
    users
    .join(users_more, "user_id", "left")
)

res.write.csv(
    "hdfs:/users_with_more",
    nullValue="\\N"
)

### Commits Punchcard for top users

In [36]:
commits = utils.read_csv(spark, "hdfs:/commits_new.csv")
projects = utils.read_csv(spark, "hdfs:/projects_new.csv")

# top_users = [5203, 896, 376498, 6240, 1779, 9236, 1570, 3871, 1736, 13009, 24452, 616741, 2468643, 2427, 81423, 796, 10005, 417948, 2016667, 1954]
jamians = [1432224, 5107602, 4007006, 6145009, 2859386, 4925305, 2549876]

res = (
    commits
    .where(
        (commits.created_at.isNotNull())
#         & (commits.author_id.isin(top_users))
        & (commits.author_id.isin(jamians))
    )
    .select(
        "author_id",
        F.date_format('created_at', 'E').alias('day'),
        F.hour('created_at').alias('hour')
    )
   .groupBy(
    'author_id',
    'day',
    'hour'
   )
   .count()
   .withColumnRenamed("count", "commits")
#    .withColumnRenamed("author_id", "user_id")
)

res.write.json(
    "hdfs:/jamians_commit_punchcard"
)

### Commits Punchcard for EVERYONE!

In [8]:
res = (
    commits
    .where(
        (commits.created_at.isNotNull())
    )
    .select(
        F.date_format('created_at', 'E').alias('day'),
        F.hour('created_at').alias('hour')
    )
   .groupBy(
    'day',
    'hour'
   )
   .count()
   .withColumnRenamed("count", "commits")
)

res.write.json(
    "hdfs:/commit_punchcard"
)

### Projects (More Data)

In [14]:
projects =  utils.read_csv(spark, "hdfs:/projects_new.csv")
stars = utils.read_csv(spark, "hdfs:/watchers.csv")
issues = utils.read_csv(spark, "hdfs:/issues.csv")

pcommits = utils.read_csv(spark, "hdfs:/project_commit_count.csv")
pcommits_others = utils.read_csv(spark, "hdfs:/project_commit_others_count.csv")

members = utils.read_csv(spark, "hdfs:/project_members_new.csv")

# Find stars on a repo
pstars = (
    stars
    .groupby("repo_id")
    .count()
    .withColumnRenamed("count", "stars")
    .withColumnRenamed("repo_id", "project_id")
)

# Find forks of a repo
pforks = (
    projects
    .where(projects.forked_from.isNotNull())
    .groupby("forked_from")
    .count()
    .withColumnRenamed("count", "forks")
    .withColumnRenamed("forked_from", "project_id")
)

# Find total issues on a repo
pissues = (
    issues
    .where(issues.pull_request_id.isNull())
    .groupby("repo_id")
    .count()
    .withColumnRenamed("count", "issues")
    .withColumnRenamed("repo_id", "project_id")
)

# Find pull requests on a repo
ppullreqs = (
    issues
    .where(issues.pull_request_id.isNotNull())
    .groupby("repo_id")
    .count()
    .withColumnRenamed("count", "pull_requests")
    .withColumnRenamed("repo_id", "project_id")
)

# Find members of a repo
pmembers = (
    members
    .groupby("project_id")
    .count()
    .withColumnRenamed("count", "contributors")
)

# Join Data
pmore = (
    pstars
    .join(pforks, "project_id", "full")
    .join(pmembers, "project_id", "full")
    .join(pissues, "project_id", "full")
    .join(ppullreqs, "project_id", "full")
    .join(pcommits, "project_id", "full")
    .join(pcommits_others, "project_id", "full")
)

# pmore.show()

pmore.write.csv(
    "hdfs:/projects_more",
    mode="overwrite",
    nullValue="\\N"
)

### Users (More Data)

In [18]:
issues = utils.read_csv(spark, "hdfs:/issues.csv")
members = utils.read_csv(spark, "hdfs:/project_members_new.csv")

user_more = utils.read_csv(spark, "hdfs:/user_more.csv")

ucommits = utils.read_csv(spark, "hdfs:/user_commit_count.csv")
ucommits_others = utils.read_csv(spark, "hdfs:/user_commit_others_count.csv")

projects = utils.read_csv(spark, "hdfs:/projects_new.csv")
projects_more = utils.read_csv(spark, "hdfs:/projects_more")

projects.createOrReplaceTempView("projects")
projects_more.createOrReplaceTempView("projects_more")

# Find total stars that a users repos have recieved
q = """
    SELECT P.owner_id as user_id, sum(M.stars) as stars_on_repos
    FROM projects as P, projects_more as M
    WHERE P.id = M.project_id
    GROUP BY P.owner_id
"""

ustars = spark.sql(q)

# Find total repos a user is members of
umembers = (
    members
    .groupby("user_id")
    .count()
    .withColumnRenamed("count", "members_of_repos")
)


# Find total issues by user
uissues = (
    issues
    .where(issues.pull_request_id.isNull())
    .groupby("reporter_id")
    .count()
    .withColumnRenamed("count", "issues")
    .withColumnRenamed("reporter_id", "user_id")
)

# Find pull requests by user
upullreqs = (
    issues
    .where(issues.pull_request_id.isNotNull())
    .groupby("reporter_id")
    .count()
    .withColumnRenamed("count", "pull_requests")
    .withColumnRenamed("reporter_id", "user_id")
)

# Join Data
umore = (
    user_more
    .join(ustars, "user_id", "full")
    .join(umembers, "user_id", "full")
    .join(uissues, "user_id", "full")
    .join(upullreqs, "user_id", "full")
    .join(ucommits, "user_id", "full")
    .join(ucommits_others, "user_id", "full")
)

umore.write.csv(
    "hdfs:/users_most",
    mode="overwrite",
    nullValue="\\N"
)

### Stars, Followers of Top Corporates

In [11]:

top_orgs = {
    "Microsoft": 8071,
    "IBM": 2804,
    "Google": 2725,
    "Red Hat": 1669,
    "Intel": 1027,
    "ThoughtWorks": 937,
    "Facebook": 925,
    "Esri": 888,
    "Tencent": 818,
    "Oracle": 672,
    "Accenture": 656,
    "Baidu": 552,
    "Amazon": 549,
    "EPAM Systems": 519,
    "Yandex": 490,
    "Capgemini": 488,
    "TCS": 475,
    "Cisco": 470,
    "Alibaba": 461,
    "Mozilla": 457,

}

top_orgs_indian = {
    "TCS": 261,
    "Microsoft": 227,
    "Accenture": 181,
    "Tata Consultancy Services": 169,
    "ThoughtWorks": 163,
    "Cognizant Technology Solutions": 163,
    "Infosys": 162,
    "Flipkart": 154,
    "IBM": 149,
    "Cognizant": 125,
    "Sapient": 119,
    "Capgemini": 109,
    "Freshdesk": 100,
    "Oracle": 99,
    "Practo": 80,
    "Wipro Technologies": 79,
    "Tech Mahindra": 72,
    "Amazon": 71,
    "Red Hat": 68,
    "Thoughtworks": 66,
}

# top_corps = list(top_orgs.keys())
top_corps = list(top_orgs_indian.keys())

MSFT = ["Microsoft Corporation", "Microsoft"]

user_more = utils.read_csv(spark, "hdfs:/users_with_more.csv")

users_of_top = (
    user_more
    
    .where(user_more.country_code == "in")
    
    .select("company", "has_stars", "followers",
            F.when(user_more.company.isin(MSFT), "Microsoft")
             .otherwise(user_more.company).name("corporate")
    )
)

users_of_top = (
    users_of_top
    .where(users_of_top.corporate.isin(top_corps))
)

users_of_top.createOrReplaceTempView("users_top")

q = """

    SELECT corporate, count(*) as employees, 
           sum(`has_stars`) as total_stars, 
           sum(`followers`) as total_followers, 
           ceiling(avg(has_stars)) as stars_per_employee, 
           ceiling(avg(followers)) as followers_per_employee
           
    FROM users_top as ut
    GROUP BY corporate
    
    ORDER BY followers_per_employee DESC
"""

res = spark.sql(q)

res.show(20, truncate=False)

+------------------------------+---------+-----------+---------------+------------------+----------------------+
|corporate                     |employees|total_stars|total_followers|stars_per_employee|followers_per_employee|
+------------------------------+---------+-----------+---------------+------------------+----------------------+
|Red Hat                       |69       |1690       |1058           |41                |25                    |
|Flipkart                      |156      |1078       |582            |27                |11                    |
|ThoughtWorks                  |163      |2536       |1018           |32                |10                    |
|Microsoft                     |235      |892        |1102           |14                |10                    |
|IBM                           |157      |1117       |431            |33                |9                     |
|Practo                        |82       |318        |209            |14                |6      

### %age Community Particitpation

In [29]:
pm = utils.read_csv(spark, "hdfs:/projects_more.csv")
proj = utils.read_csv(spark, "hdfs:/projects_new.csv")

pm = (
#     pm
    proj.join(pm, proj.id == pm.project_id, "left")
    .na.fill({'commits': 1, 'commits_by_others': 0})
)

pm.createOrReplaceTempView("projects_more")

q = """
    SELECT ceiling( 100 * (commits_by_others) / commits ) as community_part, count(*) as num_repos
    FROM projects_more
    GROUP BY community_part
    ORDER BY community_part
"""

# pm.limit(50).show(50)

res = spark.sql(q)
res.show(100)

+--------------+---------+
|community_part|num_repos|
+--------------+---------+
|             0| 51332178|
|             1|    43473|
|             2|    75228|
|             3|    78215|
|             4|    85897|
|             5|    82493|
|             6|    73771|
|             7|    70892|
|             8|    80642|
|             9|    58070|
|            10|   108058|
|            11|    28597|
|            12|    82883|
|            13|    79445|
|            14|    37588|
|            15|    98982|
|            16|    38888|
|            17|   114522|
|            18|    26443|
|            19|    44038|
|            20|   159205|
|            21|    14105|
|            22|    30787|
|            23|    49053|
|            24|    35213|
|            25|   222877|
|            26|    11828|
|            27|    26534|
|            28|    35727|
|            29|    62526|
|            30|    41438|
|            31|    23168|
|            32|    26018|
|            33|    12930|
|

### Counts

In [31]:
pm = utils.read_csv(spark, "hdfs:/projects_more.csv")
proj = utils.read_csv(spark, "hdfs:/projects_new.csv")

pm = (
    proj.join(pm, proj.id == pm.project_id, "left")
    .na.fill({'commits': 0, 'commits_by_others': 0})
)

pm.where(pm.commits == 0).count()

25922057

### Lifespan of Projects (by last-first commit)

In [40]:
cm = utils.read_csv(spark, "hdfs:/commits_new.csv")
cm.createOrReplaceTempView("commits")

q = """
    SELECT project_id,
           min(created_at) as first,
           max(created_at) as last
    FROM commits
    WHERE
      created_at < CURRENT_TIMESTAMP
      AND project_id IS NOT NULL
    GROUP BY project_id
"""

res = spark.sql(q)

res = (
    res
    .withColumn('duration', F.datediff(F.col("last"), F.col("first")))
    .select("project_id", "duration")
    .groupby("duration")
    .count()
    .sort(F.desc("duration"))
)

res.write.json(
    "hdfs:/lifespan_commits_1.json",
    mode="overwrite",
#     nullValue="\\N"
)

### Top Languages

In [42]:
    
pl = utils.read_csv(spark, "hdfs:/project_languages.csv")

res1 = (
    pl
    .select("project_id", "language", "bytes", F.year("created_at").alias("year"))
    .groupby("project_id", "language", "year")
    .max("bytes")
)

res1.show(20)

res2 = (
    res1
    .groupby("language", "year")
    .sum("bytes")
    .sort("bytes", False)
)

res2.show(20)

+----------+------------+----+----------+
|project_id|    language|year|max(bytes)|
+----------+------------+----+----------+
|  13496571|           d|2015|      3759|
|   5999634|        perl|2015|    465114|
|  11309521|      python|2015|      7790|
|  16054889|       shell|2015|       478|
|  22350750|         css|2015|       808|
|  15363845|      prolog|2015|       199|
|   6030637|       shell|2015|        64|
|  10750668|         c++|2015|     33000|
|  25330477|         css|2015|       122|
|   4650978|        java|2015|    140600|
|  19559892|         css|2015|      1717|
|  18461008|        xslt|2015|    220483|
|  21784276|         css|2015|      1934|
|  21949925|        viml|2015|     16714|
|   2529417|coffeescript|2015|     45230|
|  16543457|        haxe|2015|     11075|
|  19814407|      python|2015|    479806|
|  10522756|       shell|2015|       126|
|  14547901|        java|2015|      2507|
|  14548067|  javascript|2015|      3386|
+----------+------------+----+----

AnalysisException: 'Cannot resolve column name "bytes" among (project_id, language, year, max(bytes));'